# Лабораторная работа №1
### 1. Постановка задачи
1. На языке Python программно реализовать два метрических алгоритма классификации: Naive Bayes и K Nearest Neighbours
2. Сравнить работу реализованных алгоритмов с библиотечными из scikit-learn
3. Для тренировки, теста и валидации использовать один из предложенных датасетов (либо найти самостоятельно и внести в таблицу)
4. Сформировать краткий отчет (постановка задачи, реализация, эксперимент с данными, полученные характеристики, вывод

### 2. Исходные данные
**Датасет**: https://archive.ics.uci.edu/ml/datasets/Smartphone-Based+Recognition+of+Human+Activities+and+Postural+Transitions
<br>
**Предметная область**: Человеческая активность, различные движения
<br>**Список классов**:
1. WALKING
2. WALKING_UPSTAIRS
3. WALKING_DOWNSTAIRS
4. SITTING
5. STANDING
6. LAYING
7. STAND_TO_SIT
8. SIT_TO_STAND
9. SIT_TO_LIE
10. LIE_TO_SIT
11. STAND_TO_LIE
12. LIE_TO_STAND

**Количество атрибутов**: 561
<br>
**Основные атрибуты**: Измерения гироскопа и акселерометра в 3х осях, фильтры их значений
<br>
**Полный список атрибутов**: Features.txt
### 3. Ход работы
#### 1. Реализация алгоритма Naive Bayes.


In [ ]:
class NaiveBayes:
    summaries = {}

    # обучение классификатора
    def fit(self, train_data, train_classes):
        # получаем словарь с данными, разделенный по классам
        classes_dict = self.separate_by_class(train_data, train_classes)

        for class_name, items in classes_dict.items():
            # считаем среднее значение и среднеквадратичное отклонение атрибутов для каждого класса входных данных
            self.summaries[class_name] = self.summarize(items)

    def separate_by_class(self, train_data, train_classes):
        classes_dict = {}
        for i in range(len(train_data)):
            classes_dict.setdefault(train_classes[i], []).append(train_data[i])

        return classes_dict

    # обобщаем данные
    def summarize(self, class_data):
        summaries = [(self.mean(attributes), self.stand_dev(attributes)) for attributes in
                     zip(*class_data)]
        return summaries

    # вычисление среднего значения
    def mean(self, values):
        return sum(values) / float(len(values))

    # вычисление дисперсии
    def stand_dev(self, values):
        var = sum([pow(x - self.mean(values), 2) for x in values]) / float(len(values) - 1)
        return math.sqrt(var)

    # классификация тестовой выборки
    def predict(self, data_x):
        predictions = []
        for x in data_x:
            predictions.append(self.predict_one(self.summaries, x))
        return predictions

    # классификация одного объекта
    def predict_one(self, summaries, x):
        probabilities = self.calc_class_probabilities(summaries, x)
        best_class = None
        max_prob = -1
        for class_name, probability in probabilities.items():
            if best_class is None or probability > max_prob:
                max_prob = probability
                best_class = class_name
        return best_class

    # вычисление вероятности принадлежности объекта к каждому из классов
    def calc_class_probabilities(self, summaries, instance_attr):
        probabilities = {}
        for class_name, class_summaries in summaries.items():
            probabilities[class_name] = 1.0
            for i in range(len(class_summaries)):
                mean, stdev = class_summaries[i]
                x = float(instance_attr[i])
                probabilities[class_name] *= self.calc_probability(x, mean, stdev)
        return probabilities

    # вычисление апостериорной вероятности принадлежности объекта к определенному классу
    def calc_probability(self, x, mean, stdev):
        if stdev == 0:
            stdev += 0.000001
        exponent = math.exp(-(math.pow(x - mean, 2) / (2 * math.pow(stdev, 2))))
        return (1 / (math.sqrt(2 * math.pi) * stdev)) * exponent


#### Сравнение работы реализованного алгоритма с библиотечным:

In [ ]:
file_data, file_classes = utils.load_data()
data = Data(file_data, file_classes, 1, 0.7)

# NB
gnb = GaussianNB()
nb_clf = gnb.fit(data.train_data, data.train_classes)
accuracy = accuracy_score(data.test_classes, gnb.predict(data.test_data))
print('Naive Bayes Accuracy: %.8f' % accuracy)

# my NB
bayes_native = NaiveBayes()
bayes_native.fit(data.train_data, data.train_classes)
bayes_native_accuracy = accuracy_score(data.test_classes, bayes_native.predict(data.test_data))
print('my naive Bayes Accuracy: %.8f' % bayes_native_accuracy)

**Naive Bayes Accuracy**: 0.79613734
<br>**my naive Bayes Accuracy**: 0.78969957

#### 2. Реализация алгоритма K Nearest Neighbours

In [ ]:
class KNearestNeighbors:
    def __init__(self, neighbours_size):
        self.neighbours_size = neighbours_size

    def fit(self, train_x, train_y):
        self.train_x = train_x
        self.train_y = train_y

    # классификация тестовой выборки
    def predict(self, data_x):
        predictions = []
        for x in data_x:
            neighbours = self.get_neighbours(x)
            predicted_class = self.predict_class(neighbours)
            predictions.append(predicted_class)
        return predictions

    # находим расстояния до всех объектов
    def get_neighbours(self, inst):
        distances = []
        for i in self.train_x:
            distances.append(self.get_euclidean_distance(i, inst))
        distances = tuple(zip(distances, self.train_y))
        sorted_list = sorted(distances)[:self.neighbours_size]

        class_name_list = []
        for value, class_name in sorted_list:
            class_name_list.append(class_name)
        return class_name_list

    # евклидово расстояние между двумя объектами
    def get_euclidean_distance(self, inst1, inst2):
        distances = [(i - j) ** 2 for i, j in zip(inst1, inst2)]
        return math.sqrt(sum(distances))

    # определение самого распространенного класса среди соседей
    def predict_class(self, neighbours):
        return Counter(neighbours).most_common()[0][0]


#### Сравнение работы реализованного алгоритма с библиотечным:

In [ ]:
# KNN
sklearn_kn_clf = KNeighborsClassifier(10)
sklearn_kn_clf.fit(data.train_data, data.train_classes)
sklearn_kn_clf_accuracy = accuracy_score(data.test_classes, sklearn_kn_clf.predict(data.test_data))
print('knn Accuracy: {:.8%}'.format(sklearn_kn_clf_accuracy))

# My KNN
my_kn_clf = KNearestNeighbors(10)
my_kn_clf.fit(data.train_data, data.train_classes)
my_kn_clf_accuracy = accuracy_score(data.test_classes, my_kn_clf.predict(data.test_data))
print('my knn Accuracy: {:.8%}'.format(my_kn_clf_accuracy))


**knn Accuracy**: 90.98712446% <br>
**my knn Accuracy**: 92.06008584%

#### Вспомогательные методы\классы

In [7]:
def load_data():
    data = np.loadtxt('../HAPT Data Set/Train/X_train.txt', delimiter=' ')
    classes = np.loadtxt('../HAPT Data Set/Train/y_train.txt')
    return data, classes


class Data:
    def __init__(self, data, classes, size, train_size, attributes_count = 561):
        size = int(data.shape[0] * size)
        self.data = data[0:size, :attributes_count]
        self.classes = classes[0:size]
        self.train_data, self.test_data, self.train_classes, self.test_classes = train_test_split(self.data, self.classes,
                                                                                                  train_size=train_size,
                                                                                                  test_size=1 - train_size)

### Вывод
В ходе лабораторной работы были получены практические навыки работы с метрическими методами машинного обучения на практических примерах с использованием языка программирования python и библиотеки sklearn.
<br>
<br>
Были использованы классификаторы K Nearest Neighbor Classifier и Naive Bayes Classifier из библиотеки sklearn, а также имплиментироваы собственные реализации данных алгоритмов. Библиотечные и реализованные алгоритмы показали примерно одинаковую высокую точность предсказаний.
